In [15]:
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import random
import chardet
from time import sleep
from rescue import *

In [16]:
browser = open_browser()
browser = login(browser)

In [17]:
data = pd.read_excel("rescue.xls")
numbers = ["".join(str(i).split()) for i in data["案件編號"].values]
#Service Selection
button1 = browser.find_element_by_id("案件維護")
button1.click()

for n in range(len(numbers)):
    browser = find_page(browser, numbers[n])
    table = browser.find_element_by_name("myForm")
    table_rows = table.find_elements_by_tag_name('tr')
    table_cols = table_rows[2].find_elements_by_tag_name('td')
    button = table_cols[13].find_element_by_xpath("input[@name='Hist']")
    button.click()
    browser.switch_to.window(browser.window_handles[1])
    
    sleep(1)
    pretime = extract(browser)
    print(pretime)
    data.loc[n, "分隊車庫影像出勤時間"] = pretime
    time1 = data.loc[n, "派遣時間"]
    time2 = data.loc[n, "分隊車庫影像出勤時間"]
    data.loc[n, "出勤時間(秒)"] = int(time_conversion(time2)-time_conversion(time1))
    
    browser.switch_to.window(browser.window_handles[0])
    browser.back()

17:45:07
13:13:50
13:27:08
08:32:19
07:43:52
18:02:16
02:26:39
22:19:39
19:45:56
19:26:43
09:11:33
09:52:35
00:06:44
18:10:13
09:40:58
07:23:33
20:55:51
02:31:16
22:14:06
08:37:34
13:02:30
18:37:58
05:02:45
14:52:34
11:08:20
07:48:49
16:35:48
16:52:29
07:06:58
11:09:59
19:43:31
21:09:20
15:20:00
08:03:25
22:17:48
09:28:55
16:29:16
06:37:06
06:28:10
12:13:10
23:11:43
11:41:29
00:24:04
17:08:38


In [18]:
data.to_excel("time.xlsx")
data

,序號,案件編號,受理時間,受理時間.1,案發地點,分隊,派遣時間,分隊車庫影像出勤時間,到達現場時間,出勤時間(秒),備註
0,1,1812031743481,107-12-03,17:43:48,台北市中正區濟南路1段2-2號,華山分隊92,17:44:27,17:45:07,17:48:11,40.0,NaN
1,2,181203131103B,107-12-03,13:11:03,台北市大安區仁愛路3段18號1樓派出所內，拒送(無送醫意願),忠孝分隊91,13:11:57,13:13:50,13:19:06,113.0,NaN
2,3,1812031325202,107-12-03,13:25:20,捷運中正紀念堂站,泉州分隊93,13:25:41,13:27:08,13:32:19,87.0,NaN
3,4,1812030830553,107-12-03,08:30:55,台北市大安區基隆路4段73巷巷口,古亭分隊91,08:31:38,08:32:19,08:33:33,41.0,NaN
4,5,181203074307B,107-12-03,07:43:07,館前路 與 襄陽路 交叉路口,城中分隊91,07:43:30,07:43:52,07:50:44,22.0,NaN
5,6,181203180103B,107-12-03,18:01:03,台北市中正區西藏路35號1樓,雙園分隊91,18:01:14,18:02:16,18:05:02,62.0,NaN
6,7,181203022456A,107-12-03,02:24:56,台北市萬華區昆明街109號,龍山分隊91,02:25:27,02:26:39,02:28:52,72.0,NaN
7,8,181203221809B,107-12-03,22:18:09,台北市文山區木新路2段221號3樓,萬芳分隊93,22:18:27,22:19:39,22:23:25,72.0,NaN
8,9,1812031943324,107-12-03,19:43:32,興隆路 與 福興路 交叉路口,景美分隊91,19:44:26,19:45:56,19:48:55,90.0,NaN
9,10,181203182309A,107-12-03,17:14:09,台北市文山區新光路2段74巷11號1樓,木柵分隊91,18:26:04,19:26:43,17:24:51,3639.0,NaN
